## Goal: Label matching names
---

### Step 1: Load packages
---

In [1]:
# import sys
# sys.path.insert(0, '/healthcare-fraud/src/')
import src.scripts as src
import pandas as pd
import numpy as np

### Step 2: Load data
---

In [2]:
# load data
LEIE_id_full = src.read_from_efs('LEIE_id_full.csv')
LEIE_id_unlabeled = src.read_from_efs('LEIE_id_unlabeled.csv')
PartD_id_full = src.read_from_efs('PartD_id_full.csv')
PartD_id_unlabeled = src.read_from_efs('PartD_id_unlabeled.csv')
fuzz_scores_df = src.read_from_efs('fuzz_scores_df.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Step 3: Extract fuzzy matches
---

In [40]:
# get top pairs from each fuzzywuzzy algo
algos = ['Qratio','UQratio','UWQratio','partial_ratio','partial_token_sort_ratio','token_set_ratio','token_sort_ratio']

# reduce to top highest by 'partial ratio'
df = fuzz_scores_df[fuzz_scores_df['partial_ratio']>65].sort_values('UQratio', ascending=False).reset_index(drop=True)
df1 = df[df['partial_token_sort_ratio']>65]
fuzz_matches_df = df1[df1['token_sort_ratio']>65].reset_index(drop=True)

src.save_to_efs(fuzz_matches_df, 'fuzz_matches_df.csv')

### Step 4: Label fuzzy matches
---

In [49]:
# Label LEIE
LEIE_id_labeled_fuzzy = LEIE_id_full[LEIE_id_full['full'].isin(fuzz_matches_df['LEIE'])].reset_index(drop=True)
src.save_to_efs(LEIE_id_labeled_fuzzy,'LEIE_id_labeled_fuzzy.csv')
LEIE_id_labeled_fuzzy

,exclusion year,npi,last/org name,first name,city,specialty,full
0,2017,0,CARDENAS,CARLOS,MIAMI,PHARMACY,CARDENAS CARLOS PHARMACY
1,2016,0,GONZALEZ,ORLANDO,MIAMI,PHARMACY,GONZALEZ ORLANDO PHARMACY
2,2017,0,HERNANDEZ,FRANCISCO,MIAMI,RECIPT/BENEFICIARY,HERNANDEZ FRANCISCO RECIPT/BENEFICIARY
3,2014,0,MECHANIC,JEFFREY,NEW YORK,COUNSELOR,MECHANIC JEFFREY COUNSELOR
4,2015,0,NARCISSE,NAPHTHALIE,TAMPA,THERAPIST,NARCISSE NAPHTHALIE THERAPIST
5,2015,1881900967,PETERSEN,RONALD,CHARLESTON,COUNSELING CENTER,PETERSEN RONALD COUNSELING CENTER
6,2015,0,RIVERA,FRANCISCO,MAYAGUEZ,PHARMACY,RIVERA FRANCISCO PHARMACY
7,2018,0,WILLIAMS,WILLIAM,GREENVILLE,NURSE/NURSES AIDE,WILLIAMS WILLIAM NURSE/NURSES AIDE


In [53]:
PartD_id_full[PartD_id_full['full'].isin(fuzz_matches_df['PartD'])]

,npi,last/org name,first name,city,specialty,exclusion_method,exclusion_year,full
151779,1184852543,NARCISSE,NAPTHALIE,TAMPA,Massage Therapist,NaN,NaN,NARCISSE NAPTHALIE Massage Therapist
158677,1194830760,PETERSEN,RONALD,CHARLESTON,Dentist,NaN,NaN,PETERSEN RONALD Dentist
167826,1205918331,WILLIAMS,WILLIAM,ASHEVILLE,Dentist,NaN,NaN,WILLIAMS WILLIAM Dentist
310880,1386718096,WILLIAMS,WILLIAM,CHICAGO,Dentist,NaN,NaN,WILLIAMS WILLIAM Dentist
458623,1568649804,CARDENAS,CARLOS,MIAMI,Dentist,NaN,NaN,CARDENAS CARLOS Dentist
496440,1619282126,GONZALEZ,ORLANDO,MIAMI,Physician Assistant,NaN,NaN,GONZALEZ ORLANDO Physician Assistant
505833,1629185632,WILLIAMS,WILLIAM,KNOXVILLE,Dentist,NaN,NaN,WILLIAMS WILLIAM Dentist
592681,1730203738,RIVERA RIVERA,FRANCISCO,MAYAGUEZ,Family Practice,NaN,NaN,RIVERA RIVERA FRANCISCO Family Practice
595662,1730375973,MECHANICK,JEFFREY,NEW YORK,Endocrinology,NaN,NaN,MECHANICK JEFFREY Endocrinology
690341,1851464432,WILLIAMS,WILLIAM,GREENVILLE,Dentist,NaN,NaN,WILLIAMS WILLIAM Dentist


In [ ]:
top_fuzz_scores_df.sort_values('Qratio', ascending=False)

In [ ]:
fuzz_scores_df.boxplot()
pl